In [1]:
from google.colab import drive
from PIL import Image
import os, sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

##Path with the original images to obtain LR images.
LR_path = '/content/drive/My Drive/Curs 2022-2023/Deep Learning/FP/Data/images/train_DIV2K/Low Resolution/'
##Path with the original images to obtain HR images.
HR_path = '/content/drive/My Drive/Curs 2022-2023/Deep Learning/FP/Data/images/train_DIV2K/High Resolution/'
##Path to save the images in LR.
cropped_LR_path = '/content/drive/My Drive/Curs 2022-2023/Deep Learning/FP/Data/images/train_DIV2K/Low Resolution Results/'
##Path to save the images in HR.
cropped_HR_path = '/content/drive/My Drive/Curs 2022-2023/Deep Learning/FP/Data/images/train_DIV2K/High Resolution Results/'

##Path to save the MAT file.
mat_path = '/content/drive/My Drive/Curs 2022-2023/Deep Learning/FP/Mat/'


def resize_image(max_width, max_height, input_path, output_path):
    # Get the list of files in the input directory
    files = os.listdir(input_path)

    for file_name in files:
        # Open the image file
        image = Image.open(input_path + file_name)

        # Get the current width and height of the image
        width, height = image.size

        # Calculate the scaling factor
        scaling_factor = min(max_width/width, max_height/height)

        # Scale the image while maintaining aspect ratio
        image.thumbnail((width * scaling_factor, height * scaling_factor), Image.ANTIALIAS)

        # Get the new dimensions of the image
        width, height = image.size

        # Calculate the coordinates for cropping the center of the image
        left = (width - max_width) / 2
        top = (height - max_height) / 2
        right = (width + max_width) / 2
        bottom = (height + max_height) / 2

        # Crop the center of the image
        image = image.crop((left, top, right, bottom))

        # Generate the output file name
        file_name, file_extension = os.path.splitext(file_name)
        file_name = file_name + ".png"

        # Save the image to the output directory
        image.save(output_path + file_name, 'PNG')


def save_MAT(stop):
    # Get the list of files in the X and y directories
    dir_X = sorted(os.listdir(resultPath_X))
    dir_y = sorted(os.listdir(resultPath_y))

    boolean = True

    # Merge the X and y directories
    merged_dir = zip(dir_X, dir_y)

    for i, (item_X, item_y) in enumerate(merged_dir):
        # Read the X and y images
        image_X = cv2.imread(resultPath_X + item_X)
        image_y = cv2.imread(resultPath_y + item_y)

        # Convert the images to RGB color space
        image_X = cv2.cvtColor(image_X, cv2.COLOR_BGR2RGB)
        image_y = cv2.cvtColor(image_y, cv2.COLOR_BGR2RGB)

        # Expand dimensions to add a channel dimension
        image_X = np.expand_dims(image_X, axis=3)
        image_y = np.expand_dims(image_y, axis=3)

        if boolean:
            boolean = False
            data = image_X
            label = image_y
        else:
            # Concatenate the images along the channel dimension
            data = np.concatenate((data, image_X), axis=3)
            label = np.concatenate((label, image_y), axis=3)

        # Break the loop when the specified stop condition is met
        if i % stop == 0 and i != 0:
            break

    # Save the data and label arrays to a .mat file
    sio.savemat(mat_path + 'dataset64to256.mat', {'X': data, 'y': label})

In [ ]:
## 64x54 LR cropped images
resize(64,64,LR_path, cropped_LR_path)

## 256x256 HR cropped images
resize(256,256,HR_path, cropped_HR_path)

## Mat with the 800 images
saveMAT(800)

Creating .mat file at 0:27:51
0.0% -- 0.008954048156738281 seconds
Finished at 0:27:51
